Stel, een klant komt naar je toe en vraagt of je overeenkomsten kan vinden tussen klanten. Deze overeenkomsten moeten alle klanten groeperen om zo de groepen beter te kunnen benaderen met bv. gepersonaliseerde reclame of aanbiedingen.

Één mogelijke manier is door gebruik te maken van een clustering algoritme. Omdat de dataset klein is en verklaarbaarheid van de data een grote rol speelt, maken we gebruik van hiërarchische clustering analyse.

Laten we beginnen met het importeren van de dataset.

In [ ]:
import pandas as pd

# Substitute the path_to_file content by the path to your shopping-data.csv file 
path_to_file = 'shopping-data.csv'
customer_data = pd.read_csv(path_to_file)

In [ ]:
customer_data.shape

In [ ]:
customer_data.columns

Laten we even wat verder induiken op de "spending habits" van de klanten door de spending score in een histogram te plaatsen.

In [ ]:
customer_data['Spending Score (1-100)'].hist()

In [ ]:
# transpose() transposes the table, making it easier for us to compare values
customer_data.describe().transpose()

In [ ]:
customer_data.info()

In [ ]:
customer_data.head() 

In [ ]:
customer_data['Genre'].unique()

In [ ]:
customer_data['Genre'].value_counts(normalize=True)

In [ ]:
intervals = [15, 20, 30, 40, 50, 60, 70]
col = customer_data['Age']
customer_data['Age Groups'] = pd.cut(x=col, bins=intervals)

# To be able to look at the result stored in the variable
customer_data['Age Groups'] 

In [ ]:
customer_data.groupby('Age Groups')['Age Groups'].count()

In [ ]:
# The _oh means one-hot
customer_data_oh = pd.get_dummies(customer_data)
# Display the one-hot encoded dataframe
customer_data_oh 

In [ ]:
customer_data_oh = customer_data_oh.drop(['Age'], axis=1)
customer_data_oh.shape 

Het voordeel van one-hot-encoding is het versimpelen van het weergeven van kolom waardes. Het is makkelijk te lezen en te begrijpen. Het nadeel is dat er nu 8 EXTRA kolommen aan zijn gemaakt in de dataset. 

Hiermee kan een dendrogram gemaakt worden, maar eerst is het van belang om te kijken welke oorspronkelijke features tegenover elkaar geplot kunnen worden, waarbij clusters tevoorschijn kunnen komen.

In [ ]:
import seaborn as sns
# Seaborn is een data-visualisatie package

# Dropping CustomerID column from data 
customer_data = customer_data.drop('CustomerID', axis=1)

sns.pairplot(customer_data)

In [ ]:
sns.scatterplot(x=customer_data['Annual Income (k$)'],
                y=customer_data['Spending Score (1-100)'])

In [ ]:
import scipy.cluster.hierarchy as shc
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 7))
plt.title("Customers Dendrogram")

# Selecting Annual Income and Spending Scores by index
selected_data = customer_data_oh.iloc[:, 1:3]
clusters = shc.linkage(selected_data, 
            method='ward', 
            metric="euclidean")
shc.dendrogram(Z=clusters)
plt.show()

Het vinden van het aantal clusters is een beetje puzzelen. Er moet gekeken worden naar de grootste horizontale vlakte in een Dendrogram waar de grootste verticale lijnen door passeren. Twee mogelijke opties hierin zijn of een lijn trekken op een y van 350, of op een y van rond de 125. De lijn die door y = 350 passeert heeft echter niet de grootste verticale lijnen door zich heen lopen. Daarnaast hebben we met de Seaborn plots al een visuele indicatie van 5 clusters gezien. Dit betekent dat we een lijn trekken op y = 125.

In [ ]:
plt.figure(figsize=(10, 7))
plt.title("Customers Dendrogram with line")
clusters = shc.linkage(selected_data, 
            method='ward', 
            metric="euclidean")
shc.dendrogram(clusters)
plt.axhline(y = 125, color = 'r', linestyle = '-')


In [ ]:
from sklearn.cluster import AgglomerativeClustering

clustering_model = AgglomerativeClustering(n_clusters=7, affinity='euclidean', linkage='ward')
clustering_model.fit(selected_data)
clustering_model.labels_

In [ ]:
data_labels = clustering_model.labels_
sns.scatterplot(x='Annual Income (k$)', 
                y='Spending Score (1-100)', 
                data=selected_data, 
                hue=data_labels,
                palette="rainbow").set_title('Labeled Customer Data')